In [1]:
import pandas as pd
import math

#for apis
import requests
from urllib.parse import urlencode
import json
import base64
import webbrowser

#for working around colab not being able to open windows
from IPython.display import Javascript

searchspace_built = False

In [2]:
#spotify keys
client_id = '9bf2211d3826492aa72471ec394689e1'
client_secret = '8e28cf71dcc5438eb06e5da566a3ec28'

auth_headers = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "http://localhost:7777/callback",
    "scope": "playlist-modify-private"}

def open_web():
    url = "https://accounts.spotify.com/authorize?" + urlencode(auth_headers)
    display(Javascript('window.open("{url}");'.format(url=url)))

open_web()

<IPython.core.display.Javascript object>

In [3]:
full_callback = input("enter url: ")
code = full_callback.split("code=")[1]

encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:7777/callback"
}

r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)

print(r.json())
token = r.json()["access_token"]

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json"
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

enter url: http://localhost:7777/callback?code=AQAxQlj1CTqg99dR7RFufD4zUuwGxrK1FGDj5SgpKhKUXMCToA25hoXHuOeF2A0RWx2j6cTUxZNa5xR7r1OgzJegtiMgLiYgD5sNavFc8UdykcGCfxbONoZ3jz_T76r0dsbz_cfaLDSBIqJL6TFMliiAup-a46OFQJDokhODP3SRJTl2wxWhZXAdJEqThPMTn-ZqqDrJ6svNJXo
{'access_token': 'BQCyrnlPSwIRTFYbyJAYkx-t2R6ia4CH5PN4H9NV61TCFGrF7XzaWq10eJGx7wImbW69Q06o9XbLlqXJXEiNGTucSSBD-s83YjuVIJOnBl0R2wYPwmHXSfYwv4nycvvWWSMrbrB9JgdCicxU_zWS9bwucTosTDyaDQS8NSjH7TWme2_r03uth_JZ7ogM72fxbjvpfivAHsR3KX8', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQADnFxKol54u3aXmu5KssQ7PG-F-NHjewZ5shZ-_GPSJ1eLGR9JhS3Gu6t4MZaQSoom7A2OOpo-ZkZXSjGgLPJlQQo0BIlzlqhudSIOr3lWVWnNtgBSXpFuYr3Hpe7gtKs', 'scope': 'playlist-modify-private'}


In [ ]:
#run if you change the searchspace playlist
searchspace_built = False

In [ ]:

################################################################################

searchspace_playlist_id = '6y8vuKAhLZtk2l4UOzrg13'
seed_playlist_id = '3Z5KkzqjSL4g5A6MQ2dQ11'
seed_size = 1
recommendation_size = 75   # seed_size*rec_size < 100 if avg mode OFF
average_mode = True      # rec_size < 100 if avg mode ON
condensed_mode = True

################################################################################

def get_features(id_string, songs):

    feature_endpoint = 'audio-features?ids={}'.format(id_string)
    features_request = requests.get(BASE_URL + feature_endpoint,
                    headers=headers)
    features_json = features_request.json()

    for i in range(len(features_json['audio_features'])):

      entry = features_json['audio_features'][i]
      song = {
          'id': entry['id'],
          'danceability': entry['danceability'],
          'energy': entry['energy'],
          'valence': entry['valence'],
          'acousticness': entry['acousticness'],
          'speechiness': entry['speechiness'],
          'instrumentalness': entry['instrumentalness'],
          'liveness': entry['liveness'],
          'uri': entry['uri']
          }
      songs.append(song)

    return songs

def euclidean_distance(tuple1, tuple2):
    distance = 0
    for i in range(len(tuple1)):
        distance += (tuple1[i] - tuple2[i])**2
    return math.sqrt(distance)

def manhattan_distance(tuple1, tuple2):
    distance = 0
    for i in range(len(tuple1)):
        distance += abs(tuple1[i] - tuple2[i])
    return distance

def closest_matches(tuple_to_compare, df, n, seed_ids):

  euclideans = []
  manhattans = []
  for i in range(len(df)):

      c = df.iloc[i]
      candidate_id = c.id

      if candidate_id not in seed_ids:

        candidate = (c.danceability, c.energy, c.valence, c.acousticness,
                    c.speechiness, c.instrumentalness, c.liveness)

        candidate_uri = c.uri

        distance = euclidean_distance(candidate, tuple_to_compare)
        euclideans.append((distance, candidate_uri))
        if len(euclideans) > n:
            euclideans.sort(key=lambda x: x[0])
            euclideans.pop(-1)

        distance = manhattan_distance(candidate, tuple_to_compare)
        manhattans.append((distance, candidate_uri))
        if len(manhattans) > n:
            manhattans.sort(key=lambda x: x[0])
            manhattans.pop(-1)

  return(euclideans, manhattans)


##pull songs for search space
if searchspace_built == False:
  ids = []
  offset = 0
  while True:
    playlist_endpoint = 'playlists/{}/tracks?limit=50&offset={}'.format(searchspace_playlist_id, offset)
    playlist_request = requests.get(BASE_URL + playlist_endpoint,
                    headers=headers)
    playlist_json = playlist_request.json()

    for i in range(len(playlist_json['items'])):

      track_id = playlist_json['items'][i]['track']['id']
      ids.append(track_id)

    offset = len(ids)

    if len(ids)%50 != 0:
      break

  songs = []

  while ids:
    hundred = ids[:100]
    ids = ids[100:]
    id_string = ','.join(str(item) for item in hundred)

    songs = get_features(id_string, songs)

  df = pd.DataFrame(songs)
  searchspace_built = True


#pull data for seeds
target_ids = []
target_uris = []

offset = 0
while True:
  playlist_endpoint = 'playlists/{}/tracks?limit=50&offset={}'.format(seed_playlist_id, offset)
  playlist_request = requests.get(BASE_URL + playlist_endpoint,
                  headers=headers)
  playlist_json = playlist_request.json()

  for i in range(len(playlist_json['items'])):

    track_id = playlist_json['items'][i]['track']['id']
    target_ids.append(track_id)

    track_uri = playlist_json['items'][i]['track']['uri']
    target_uris.append(track_uri)

  offset = len(target_ids)
  if len(target_ids)%50 != 0:
    break

seed_ids = target_ids[-seed_size:]
target_uris = target_uris[:-seed_size]

seed_tuples = []
seed_songs = []
id_string = ','.join(str(item) for item in seed_ids)
seed_songs = get_features(id_string, seed_songs)
for song in seed_songs:
  seed_tuples.append((song['danceability'], song['energy'], song['valence'], song['acousticness'],
                   song['speechiness'], song['instrumentalness'], song['liveness']))

#calculate average of seed features
if average_mode:
  denom = len(seed_tuples)
  numerator = (0,0,0,0,0,0,0)
  for entry in seed_tuples:
    numerator = tuple(map(lambda x, y: x + y, numerator, entry))
  average_features = tuple(feature/denom for feature in numerator)
  seed_tuples = []
  seed_tuples.append(average_features)

#calculate recommendations
euclideans_to_add = []
manhattans_to_add = []

for tup in seed_tuples:
  euclideans, manhattans = closest_matches(tup, df, recommendation_size, seed_ids)
  for song in euclideans:
    euclideans_to_add.append(song[1])

  for song in manhattans:
    manhattans_to_add.append(song[1])

all_to_add = euclideans_to_add + manhattans_to_add
if condensed_mode:
  all_to_add = set(all_to_add)
add_uri_string = ','.join(str(item) for item in all_to_add)

uris_for_body = []
for uri in target_uris:
  uris_for_body.append({"uri": uri})

#delete old playlist, besides seed songs
delete_body = {"tracks": uris_for_body}

delete_endpoint = 'playlists/{}/tracks'.format(seed_playlist_id)
delete_request = requests.delete(BASE_URL + delete_endpoint,
                  headers=headers, json = delete_body)

#add new recommendations
add_playlist_endpoint = 'playlists/{}/tracks?uris={}'.format(seed_playlist_id,add_uri_string)
add_playlist_request = requests.post(BASE_URL + add_playlist_endpoint,
                  headers=headers)
add_playlist_json = add_playlist_request.json()

In [ ]:
add_uri_string

'spotify:track:1aclGG6ou5gG9PvZOyD322,spotify:track:3uZXyOAV88rJX1AngYuvM4,spotify:track:7vesMxBn2ojIbBwSWNV6iX,spotify:track:5Rpef3t6UoY6ohG8oZT4J6,spotify:track:3hkN8WrACZ1D81eLIFmyWI,spotify:track:2uujJLrRplaQMkEx8CybZ8,spotify:track:1KPSabIlKhWPTS695gqwjA,spotify:track:6bwgqPyy2RQeHclgHTbePg,spotify:track:2JUqYobT8NvARdPmc4ES2x,spotify:track:5sW2Dru0pxBshENdI9eOsI,spotify:track:5UBL2GPGuKPJMpdhBtbLue,spotify:track:6XCtYgE9ZOM9yyvgb7En2Q,spotify:track:294r8rdOqr0Nv1vuV5u8Qv,spotify:track:6IG2Gz1o84JKJD9Mbf3bST,spotify:track:3fORKW7mz4jD3VaKdgrAlq,spotify:track:2ERh7IJFenKz1FVWCzLKvj,spotify:track:5IWW129DwGyMVQAbaJz3rS,spotify:track:1xGrdqclNv1YXyvM28iMmD,spotify:track:2TjXFgo8V7BBfJ4HZw0z2W,spotify:track:0v56EFhzpi5Gs5zeGx3IGO,spotify:track:39oeKRgsLeynDIbWuXyA47,spotify:track:6EiTXAOhAiZuBZbW0bX0kG,spotify:track:531MBHrtGF6nAkK4zUafi9,spotify:track:0l9Z8Y8cQ6ju850iAHwcxw,spotify:track:3CyT8BLix5qsYdCx45YxQq,spotify:track:53oZXP24LECwD6YBybecvG,spotify:track:5op4axFGZZQAjig0WhM4Z0,